In [21]:
import numpy as np
import pandas
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

In [2]:
#1. Загрузите выборку из файла gbm-data.csv с помощью pandas и преобразуйте ее в массив numpy 
# (параметр values у датафрейма). В первой колонке файла с данными записано, была или нет реакция. 


In [4]:
data = pandas.read_csv('gbm-data.csv')
data[:2]

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0


In [11]:
y = data.loc[:,'Activity'].values
y[:5]

array([1, 1, 1, 1, 0])

In [ ]:
# Все остальные колонки (d1 - d1776) содержат различные характеристики молекулы, такие как размер, форма и т.д. 

In [12]:
X = data.loc[:,'D1':'D1776'].values
X[:2]

array([[0.        , 0.49700901, 0.1       , ..., 0.        , 0.        ,
        0.        ],
       [0.36666667, 0.60629148, 0.05      , ..., 0.        , 1.        ,
        0.        ]])

In [15]:
# Разбейте выборку на обучающую и тестовую, используя функцию train_test_split с параметрами test_size = 0.8 
#и random_state = 241.
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.8, random_state=241)

In [45]:
def sigmoid(y_pred):
    return 1.0 / (1.0 + np.exp(-y_pred))

In [53]:
def plot(train_loss, test_loss, file_name):
    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt
    # %matplotlib inline
    plt.figure()
    plt.plot(test_loss, 'r', linewidth=2)
    plt.plot(train_loss, 'g', linewidth=2)
    plt.legend(['test', 'train'])
    plt.savefig(file_name)

In [56]:
# 2. Обучите GradientBoostingClassifier с параметрами n_estimators=250, verbose=True, random_state=241 и 
# для каждого значения learning_rate из списка [1, 0.5, 0.3, 0.2, 0.1] проделайте следующее:
from sklearn.ensemble import GradientBoostingClassifier

min_losses={}
list = [1, 0.5, 0.3, 0.2, 0.1]
#list = [1]
for learning_rate in list:
    clf = GradientBoostingClassifier(n_estimators=250, learning_rate=learning_rate, verbose=True, random_state=241)
    clf.fit(X_train, y_train)
    # Используйте метод staged_decision_function для предсказания качества на обучающей и 
    # тестовой выборке на каждой итерации.
    train_pred_iters = clf.staged_decision_function(X_train) #staged_predict_proba
    test_pred_iters = clf.staged_decision_function(X_test)
    # Преобразуйте полученное предсказание с помощью сигмоидной функции по формуле 
    # 1 / (1 + e^{−y_pred}), # где y_pred — предсказанное значение.
    
    #Вычислите и постройте график значений log-loss (которую можно посчитать с помощью 
    # функции sklearn.metrics.log_loss) на обучающей и тестовой выборках,
    train_loss = [ log_loss(y_train, sigmoid(pred)) for pred in train_pred_iters]
    test_loss = [ log_loss(y_test, sigmoid(pred)) for pred in test_pred_iters]
    best_iter = np.argmin(test_loss)
    #а также найдите минимальное значение метрики и номер итерации,на которой оно достигается.
    min_losses[learning_rate] = (test_loss[best_iter], best_iter)
    plot(train_loss, test_loss, 'plots/%.1f.png' % (learning_rate))





      Iter       Train Loss   Remaining Time 
         1           1.0190           11.28s
         2           0.9192           11.25s
         3           0.8272            9.86s
         4           0.7834            9.00s
         5           0.7109            8.75s
         6           0.6368            8.99s
         7           0.5797            8.88s
         8           0.5610            8.45s
         9           0.5185            8.25s
        10           0.4984            7.97s
        20           0.1999            7.61s
        30           0.1313            6.96s
        40           0.0790            6.75s
        50           0.0511            6.36s
        60           0.0352            5.99s
        70           0.0245            5.58s
        80           0.0162            5.23s
        90           0.0114            4.88s
       100           0.0077            4.57s
       200           0.0004            1.37s
      Iter       Train Loss   Remaining Time 
        

In [57]:
min_losses

{0.1: (0.5269201872275844, 51),
 0.2: (0.531094637596885, 36),
 0.3: (0.5423141110024554, 10),
 0.5: (0.5582025523164261, 6),
 1: (0.5822942594278476, 0)}

In [67]:
for i in list:
    print ("%.2f, %.2f, %d" % (i, min_losses[i][0],min_losses[i][1]))

1.00, 0.58, 0
0.50, 0.56, 6
0.30, 0.54, 10
0.20, 0.53, 36
0.10, 0.53, 51


In [68]:

# 3. Как можно охарактеризовать график качества на тестовой выборке, 
# начиная с некоторой итерации: переобучение (overfitting) или недообучение (underfitting)? 
# -> overfitting

In [22]:
# 4. Приведите минимальное значение log-loss на тестовой выборке и номер итерации,
# на котором оно достигается, при learning_rate = 0.2.
#0.2: (0.531094637596885, 36),

In [69]:
# 5. 5. На этих же данных обучите RandomForestClassifier с количеством деревьев, 
# равным количеству итераций, на котором достигается наилучшее качество у градиентного 
# бустинга из предыдущего пункта, c random_state=241 и остальными параметрами по умолчанию. 
# Какое значение log-loss на тесте получается у этого случайного леса? 
# (Не забывайте, что предсказания нужно получать с помощью функции predict_proba. 
# В данном случае брать сигмоиду от оценки вероятности класса не нужно)

In [70]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=min_losses[0.2][1], random_state=241)
rf.fit(X_train, y_train)
rf_pred = rf.predict_proba(X_test)[:, 1]
rf_score = log_loss(y_test, rf_pred)

In [71]:
print ("%.2f" % rf_score)

0.54
